In [1]:
import numpy as np
import pandas as pd

In [2]:
def donchian(price, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame()
    df['upr'] = price.high.rolling(period).max().shift(periods=1)
    df['lwr'] = price.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df


def signal(price, donchian):
    '''Calculate buy & sell signals.'''
    df = pd.DataFrame(index=price.index)
    df['buy'] = np.where(price.close > donchian.upr, 1, 0)
    df['sell'] = np.where(price.close < donchian.mid, 1, 0)
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'buy'] == 1 and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'sell'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df

def weekly(exchange, tidm):
    '''Generate weekly prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    return df


def remainder_zero(series, divisor):
    '''Increment number until remainder is zero.'''
    def increment_dividend(dividend, divisor):
        '''Increment dividend while remainder does not equal zero.'''
        while dividend % divisor != 0:
            dividend += 1
        return dividend
    series = series.map(lambda x: increment_dividend(x, divisor))
    return series

In [3]:
# Input variables.
exchange = 'LSE'
tidm = 'HSV'
system = pd.Series({'1': 48, '2': 24, '3': 12, '4': 6})  # System look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Derived variables.
n = system.size # Number of systems.

In [5]:
# Import weekly prices.
price = weekly(exchange, tidm)

In [6]:
# System 1 Donchian channel.
p1 = system.loc['1']
dc1 = donchian(price, p1)

In [7]:
# Buy & sell signals.
s1 = signal(price, dc1)
s1

NameError: name 'period' is not defined

In [ ]:
# # Entry & exit signals.
# df['buy'] = np.where(df.close > df.upr, 1, 0)
# df['sell'] = np.where(df.close < df.mid, 1, 0)
# df['state'] = 0
# for i in range(period, len(df)):
#     if df.loc[df.index[i], 'buy'] == 1 and df.loc[df.index[i - 1], 'state'] == 0:
#         df.loc[df.index[i], 'state'] = 1
#     elif df.loc[df.index[i], 'sell'] == 1:
#         df.loc[df.index[i], 'state'] = 0
#     else:
#         df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
# df['entry'] = np.where(np.logical_and(df.state == 1, df.state.shift(periods=1) == 0), 1, 0)
# df['exit'] = np.where(np.logical_and(df.state == 0, df.state.shift(periods=1) == 1), 1, 0)

In [ ]:
# # Position Size.
# db = pd.DataFrame(df[df.entry == 1])
# db['volatility'] = abs((db.mid - db.close) / db.close)
# db['risk_amt'] = (position_size * risk_pct) / db.volatility
# db['shares'] = (db.risk_amt / db.close).astype('int')
# (
#     db.style
#         .format_index(lambda s: s.strftime("%Y-%m-%d"))
#         .format({
#             'open': '{:.3f}',
#             'high': '{:.3f}',
#             'low': '{:.3f}',
#             'close': '{:.3f}',
#             'upr': '{:.3f}',
#             'lwr': '{:.3f}',
#             'mid': '{:.3f}',
#             'volatility': '{:.1%}',
#             'risk_amt': '{:,.2f}',
#         })
# )